<a href="https://colab.research.google.com/github/stefano-clementini/code-sc/blob/main/06_pythonavanzato_clementini_stefano_68e0d1caf34e34381ea45e4a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Un Sistema per Sincronizzare i File tra Due Cartelle
## Caso d'Uso Aziendale: SyncEase
### Introduzione all'Azienda
SyncEase è una società tecnologica che sviluppa soluzioni software per migliorare la gestione e la condivisione dei dati aziendali. Una delle loro sfide principali è quella di garantire che i file critici siano sempre sincronizzati tra dispositivi e server, per evitare perdite di dati o versioni obsolete.

### Problema
Molte aziende si affidano a sistemi manuali o software non ottimizzati per mantenere sincronizzati i file tra due cartelle, ad esempio tra un server locale e uno di backup. Questo approccio è soggetto a errori, come la perdita di file aggiornati o la duplicazione di dati. Inoltre, il processo può essere lento, specialmente quando il volume di dati è elevato.

### Obiettivo del Progetto
L'obiettivo è sviluppare un sistema software che utilizzi il multiprocessing e il multithreading in Python per sincronizzare in modo efficiente i file tra due cartelle. Questo sistema dovrà:



1.   Copiare i file nuovi o modificati dalla cartella sorgente a quella di destinazione.
2.   Eliminare i file obsoleti nella cartella di destinazione se non esistono più nella sorgente.
3.   Gestire grandi volumi di file in modo rapido ed efficiente, sfruttando al meglio le risorse hardware.

### Benefici Attesi
*  Riduzione degli errori: Sincronizzazione affidabile e automatica senza
intervento manuale.
*  Ottimizzazione delle prestazioni: Utilizzo efficiente di CPU e I/O grazie al multiprocessing e al multithreading.
*  Miglioramento della sicurezza: Backup sempre aggiornati e consistenti tra le due cartelle.

# Specifiche del Progetto
1.   Input:
Due percorsi di cartelle: source_folder e destination_folder.
2.   Output:
La cartella di destinazione è sincronizzata con quella sorgente.
3.   Funzionalità Chiave:
*  Utilizzo di multithreading per accelerare la lettura e la scrittura dei file.
*  Utilizzo di multiprocessing per gestire in parallelo sottogruppi di file.
*  Verifica di:
   *    File nuovi: Copia solo i file che non esistono nella cartella di destinazione.
   *    File modificati: Aggiorna i file nella destinazione se quelli nella sorgente sono più recenti.
   *    File obsoleti: Rimuove i file nella destinazione che non esistono più nella sorgente.

## Soluzione:
L'applicazione creata esegue una copia di file da una cartella sorgente ad una di destinazione, definite dall'operatore.
La cartella sorgente deve essere caricata nella sezione File prima di avviare l'appliazione. La cartella di destinazione può essere caricata dall'operatore o creata in runtime.<br>
Come soluzione si propone una cartella di destinazione che copi i file su un unico livello, copiando anche i file delle sotto-cartelle di origine nello stesso percorso, per evitare più versioni dello stesso file e mantenere solo quella più recente.<br>
La funzione sync_folder ha il compito di copiare tutti i file di una cartella di origine in una di destinazione, mentre la funzione sync_folder_remove rimuove i file dalla cartella di destinazione se non sono presenti in quella di origine.<br>
L'applicazione verifica quanti CPU sono disponibili per implementare il multiprocessing. Ogni processo esegue una lista di threads. <br>
L'applicazione è multithreading perchè permette di associare la copia di ogni file ad un thread.<br>
La CPU a disposizione ha solo 2 CPU, pertanto il multiprocessing è apprezzabile solo per cartelle con un numero elevato di files.<br>
Per ogni file della cartella sorgente si verifica se il file esiste già nella cartella di destinazione e, eventualmente, se è obsoleto.<br>
Avrei potuto usare la libreria shutil per la copia dei file ma ho optato per una soluzione più scolastica di lettura-scrittura, in quanto shutil non è stata introdotta durante il corso.

In [2]:
from sys import exception
from os.path import isdir
import os
import threading
from threading import Thread
import time
import logging
import zipfile
from multiprocessing import Process
import multiprocessing
import requests
import io


def setup_logging():
  """
  Setup del logger
    il logger viene creato con 2 hanler:
    - file_handler che scrive su file di log
    - console_handler che scrive su console
  :return: logger
  """
  logger = logging.getLogger()
  logger.setLevel(logging.DEBUG)
  file_handler = logging.FileHandler('sync_log.txt')
  console_handler = logging.StreamHandler()
  formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
  file_handler.setFormatter(formatter)
  # il livello di scrittura su file è DEBUG
  file_handler.setLevel(logging.DEBUG)
  logger.addHandler(file_handler)
  console_handler.setFormatter(formatter)
  # il livello di scrittura a console è INFO
  console_handler.setLevel(logging.INFO)
  logger.addHandler(console_handler)
  return logger

def example_file(logger):
  url = "https://raw.githubusercontent.com/stefano-clementini/code-sc/main/images.zip"
  filename = 'images.zip'
  # No need for query_parameters here as we are directly accessing the raw file
  # eseguire la get per scaricare il file di esempio da github
  response = requests.get(url, stream=True)
  logger.debug(f"GET URL status: {response.status_code}")
  if response.status_code == 200:
    logger.debug("Example file Download completato")
    # write file in local filesystem
    zip_file = open(filename,"wb")
    zip_file.write(response.content)
    zip_file.close()
  else:
    logger.error("Example file  Download fallito")
    return
  # si pulisce l'ambiente
  clean_env()
  # il file 'images.zip' viene unzippato
  with zipfile.ZipFile(filename, 'r') as zip_ref:
    logger.info(f"Unzip del file 'images.zip': {zip_ref.infolist()}") # Call .infolist() as a function
    zip_ref.extractall('.')


def removedir(folder):
  """
  Eliminazione di una cartella e tutti i suoi contenuti
  :param folder: percorso della cartella da eliminare
  """
  if os.path.exists(folder):
    backup_files = os.listdir(folder)
    logger.info(f"Eliminazione file da cartella {folder}: {backup_files}")
    for backup_file in backup_files:
      if os.path.isfile(os.path.join(folder, backup_file)):
        # se è un file viene eliminato
        os.remove(os.path.join(folder, backup_file))
        logger.debug(f"File '{backup_file}' eliminato dalla cartella {folder}")
      else:
        # se è una directory chiama ricorsivamente la funzione
        removedir(os.path.join(folder, backup_file))
        logger.debug(f"Cartella '{backup_file}' eliminata dalla cartella {folder}")
    # rimuovo la cartella
    os.rmdir(folder)
    logger.debug(f"Cartella '{folder}' eliminata")

def clean_env():
  """
  Eliminazione cartelle temporanee
  """
  removedir('backup')
  removedir('folder1')
  removedir('folder2')
  removedir('folder3')

def file_list(folder):
  """
  funzione che restiruisce la lista di file, compresi i file contenuti in sotto-cartelle
  :param folder: percorso della cartella da analizzare
  :return: lista di file
  """
  dir_list = os.listdir(folder)
  out_list = []
  for i in range(len(dir_list)):
    if os.path.isdir(os.path.join(folder, dir_list[i])):
      subdir_list = file_list(os.path.join(folder, dir_list[i]))
      out_list.extend(subdir_list)
    else:
      out_list.append(dir_list[i])
  return out_list

def copy_file(source_file, destination_file, source_folder, destination_folder, logger):
  """
  Funzione che copia un file da una cartella sorgente a una cartella di destinazione
  :param source_file: nome del file da copiare
  :param destination_file: nome del file da copiare
  :param source_folder: percorso della cartella sorgente
  :param destination_folder: percorso della cartella di destinazione
  """
  try:
    logger.debug(f"Copia del file '{source_file}' da '{source_folder}' a '{destination_folder}'")
   # Apre il file sorgente in modalità lettura binaria (rb)
    # e il file di destinazione in modalità scrittura binaria (wb)
    with open(os.path.join(source_folder, source_file), 'rb') as sfile:
      with open(os.path.join(destination_folder, destination_file), 'wb') as dfile:
        logger.debug(f"Copia del file '{source_file}' da '{source_folder}' a '{destination_folder}' : Inizio")
        start_time = time.time()
        # Legge il contenuto e lo scrive nel file di destinazione
        content = sfile.read()
        dfile.write(content)
        end_time = time.time()
        logger.debug(f"Copia del file '{source_file}' da '{source_folder}' a '{destination_folder}' : Fine")
        logger.info(f"File '{source_file}' copiato da '{source_folder}' a '{destination_folder}' in {(end_time-start_time)*1000:.3f} msec")
  except FileNotFoundError:
    logger.error(f"File sorgente '{source_file}' non trovato nella cartella '{source_folder}'")
  return

def avvia_threads(threads):
  """
  Funzione che avvia i thread
  :param threads: lista di thread da avviare
  """
  for thread in threads:
    # Avvia tutti i thread del processo
    thread.start()
  for thread in threads:
    # Attende che tutti i thread del processo terminino
    thread.join()
  return

def sync_folders(source_folder, destination_folder, logger, process_number):
  """
  Funzione che sincronizza due cartelle
  :param source_folder: percorso della cartella sorgente
  :param destination_folder: percorso della cartella di destinazione
  :param logger: logger
  :param process_number: numero di processi da utilizzare
  """
  logger.info(f"Inizio sincronizzazione tra '{source_folder}' e '{destination_folder}'")
  try:
    # elementi contenuti nella cartella sorgente
    source_elements = os.listdir(source_folder)
  except FileNotFoundError:
    print(f"Cartella sorgente non trovata: {source_folder}")
    return
  logger.debug(f"Elementi nella cartella sorgente: {source_elements}")
  if not os.path.exists(destination_folder):
    # se la cartella di destinazione non esiste viene creata
    logger.debug(f"Cartella di destinazione {destination_folder} non trovata: Viene creata")
    os.makedirs(destination_folder, exist_ok=True)
  # elementi contenuti nella cartella di destinazione
  destination_elements = os.listdir(destination_folder)
  logger.debug(f"Elementi nella cartella di destinazione: {destination_elements}")
  # threads è una lista di process_number elementi
  # ogni elemento è una lista di threads
  threads = []
  for i in range(process_number):
    threads.append([])
  thread_index = 0
  for element in source_elements:
    if os.path.isdir(os.path.join(source_folder, element)):
      # se l'elemento è una directory chiama ricorsivamente la funzione
      logger.debug(f"Cartella trovata: {os.path.join(source_folder, element)}")
      sync_folders(os.path.join(source_folder, element), destination_folder, logger, process_number)
    else:
      # verifica se il file esiste già ella cartella di destinazione
      if element not in destination_elements:
        logger.debug(f"File '{element}' da copiare da '{source_folder}' a '{destination_folder}': File nuovo")
        t = threading.Thread(target=copy_file, args=(element, element, source_folder, destination_folder, logger), kwargs={})
        threads[thread_index % process_number].append(t)
        thread_index += 1
      elif  os.path.getmtime(os.path.join(source_folder, element)) > os.path.getmtime(os.path.join(destination_folder, element)):
        # verifica se il file sorgente è stato aggiornato più recentemente rispetto qullo di destinazione
        logger.debug(f"File '{element}' da copiare da '{source_folder}' a '{destination_folder}': File aggionato")
        logger.debug(f"mtime di '{os.path.join(source_folder, element)}': {os.path.getmtime(os.path.join(source_folder, element))}")
        logger.debug(f"mtime di '{os.path.join(destination_folder, element)}': {os.path.getmtime(os.path.join(destination_folder, element))}")
        t = threading.Thread(target=copy_file, args=(element, element, source_folder, destination_folder, logger), kwargs={})
        threads[thread_index % process_number].append(t)
        thread_index += 1
      else:
        logger.debug(f"File '{element}' da non copiare da '{source_folder}' a '{destination_folder}': File obsoleto")

  # Starts threads
  for process_index in range(process_number):
    proc = Process(target=avvia_threads, args=(threads[process_index],))
    proc.daemon = True
    logger.debug(f"Avvio processo {process_index}: {proc.name}")
    proc.start()

  for process_index in range(process_number):
    proc.join()
  logger.info(f"Fine sincronizzazione tra '{source_folder}' e '{destination_folder}'")

  return

def sync_folders_remove(source_folder, destination_folder, logger):
  logger.info(f"Inizio sincronizzazione tra '{destination_folder}' e '{source_folder}' per eliminare file non più presenti")
  try:
    source_elements = file_list(source_folder)
  except FileNotFoundError:
    print(f"Cartella sorgente non trovata: {source_folder}")
    return
  logger.debug(f"Elementi nella cartella sorgente: {source_elements}")
  destination_elements = file_list(destination_folder)
  logger.debug(f"Elementi nella cartella di destinazione: {destination_elements}")
  deleted_files = False
  for element in destination_elements:
    if element not in source_elements:
      logger.debug(f"File '{element}' da eliminare da '{destination_folder}': File non presente in '{source_folder}'")
      os.remove(os.path.join(destination_folder, element))
      logger.debug(f"File '{element}' eliminato da '{destination_folder}'")
      deleted_files = True
  if not deleted_files:
    logger.info(f"Nessun file da eliminare da '{destination_folder}'")
  return

if __name__ == "__main__":
  logger = setup_logging()
  example_file(logger)
  try:
    local_files = os.listdir('.')
    # dalla lista si rimuovo il file di log e i file nascosti
    if 'sync_log.txt' in local_files:
      local_files.remove('sync_log.txt')
    if '.config' in local_files:
      local_files.remove('.config')
    if '.ipynb_checkpoints' in local_files:
      local_files.remove('.ipynb_checkpoints')
    logger.info(f"Lista file nella cartella corrente: {local_files}")
    if local_files == []:
      # è necessario caricare la cartella di origine prima di eseguire l'appliazione
      logger.info("Cartella corrente vuota. E' necessario inserire cartella da archiviare")
      exit()
    # si estra il numero di CPU a disposizione
    cpu_number = multiprocessing.cpu_count()
    logger.info(f"Numero di core del computer: {cpu_number}")
    process_number = 0
    while process_number == 0:
      process_number = int(input(f"Quanti prcessi si desidera usare? [1-{cpu_number}]"))
      if process_number < 1 or process_number > cpu_number:
        logger.error(f"Il numero di processi deve essere compreso tra 1 e {cpu_number}")
        process_number = 0
    # 'images.zip' è un file che permette di verificare tutte le funzionalità richieste
    # folder1 contiene 5 elementi
    # folder2 contiene nuovi elementi. inoltre ha una sotto-cartella 'panda' con altri file
    # folder3 contiene nuovi elementi. alcuni file in backup non sono presenti in folder3 pertanto vengono rimossi
    if "images.zip" in local_files:
      use_zip = input("Si desidera usare il materiale d'esempio o inserire i path? [Y/N]")
      if use_zip == 'y' or use_zip ==  'Y':
        # sync folder1 --> backup
        source_folder = 'folder1'
        destination_folder = 'backup'
        start_time = time.time()
        sync_folders(source_folder, destination_folder, logger, process_number)
        sync_folders_remove(source_folder, destination_folder, logger)
        end_time = time.time()
        logger.info(f"Tempo di esecuzione sync {source_folder} - {destination_folder}: {(end_time-start_time)*1000:.3f} msec")
        # sync folder2 --> backup
        source_folder = 'folder2'
        destination_folder = 'backup'
        start_time = time.time()
        sync_folders(source_folder, destination_folder, logger, process_number)
        sync_folders_remove(source_folder, destination_folder, logger)
        end_time = time.time()
        logger.info(f"Tempo di esecuzione sync {source_folder} - {destination_folder}: {(end_time-start_time)*1000:.3f} msec")
        # sync folder2 --> backup
        source_folder = 'folder3'
        destination_folder = 'backup'
        start_time = time.time()
        sync_folders(source_folder, destination_folder, logger,process_number)
        sync_folders_remove(source_folder, destination_folder, logger)
        end_time = time.time()
        logger.info(f"Tempo di esecuzione sync {source_folder} - {destination_folder}: {(end_time-start_time)*1000:.3f} msec")
    else:
      source_folder = None
      while source_folder == None:
        # si inserisce la cartella di origine
        source_folder = input("Inserisci il path della cartella sorgente: ")
        if not os.path.exists(source_folder):
          # la cartella non è presente pertanto si ripete l'input
          logger.error(f"Cartella sorgente non trovata: {source_folder}")
          source_folder = None
      destination_folder = input("Inserisci il path della cartella di destinazione: ")
      print(f"source_folder: {source_folder}")
      print(f"destination_folder: {destination_folder}")
      start_time = time.time()
      sync_folders(source_folder, destination_folder, logger, process_number)
      sync_folders_remove(source_folder, destination_folder, logger)
      end_time = time.time()
      logger.info(f"Tempo di esecuzione sync {source_folder} - {destination_folder}: {(end_time-start_time)*1000:.3f} msec")
  except Exception as e:
    # si scrive nei log l'eccezione che si è verificata
    logger.error(e)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /stefano-clementini/code-sc/main/images.zip HTTP/1.1" 200 2200874
DEBUG:root:GET URL status: 200
DEBUG:root:Example file Download completato
INFO:root:Unzip del file 'images.zip': [<ZipInfo filename='folder1/' external_attr=0x10>, <ZipInfo filename='folder1/bunny.jpg' compress_type=deflate external_attr=0x20 file_size=5727 compress_size=5691>, <ZipInfo filename='folder1/dog.jpg' compress_type=deflate external_attr=0x20 file_size=29640 compress_size=29476>, <ZipInfo filename='folder1/elephant.jpg' compress_type=deflate external_attr=0x20 file_size=21993 compress_size=20465>, <ZipInfo filename='folder1/film.png' compress_type=deflate external_attr=0x20 file_size=3469 compress_size=3474>, <ZipInfo filename='folder1/fox.jpg' compress_type=deflate external_attr=0x20 file_size=53904 compress_size=36807>, <ZipInfo filename='folde

Quanti prcessi si desidera usare? [1-2]1
Si desidera usare il materiale d'esempio o inserire i path? [Y/N]y


INFO:root:Inizio sincronizzazione tra 'folder1' e 'backup'
2026-01-11 15:03:47,274 - INFO - Inizio sincronizzazione tra 'folder1' e 'backup'
2026-01-11 15:03:47,274 - INFO - Inizio sincronizzazione tra 'folder1' e 'backup'
DEBUG:root:Elementi nella cartella sorgente: ['dog.jpg', 'fox.jpg', 'elephant.jpg', 'film.png', 'bunny.jpg']
DEBUG:root:Cartella di destinazione backup non trovata: Viene creata
DEBUG:root:Elementi nella cartella di destinazione: []
DEBUG:root:File 'dog.jpg' da copiare da 'folder1' a 'backup': File nuovo
DEBUG:root:File 'fox.jpg' da copiare da 'folder1' a 'backup': File nuovo
DEBUG:root:File 'elephant.jpg' da copiare da 'folder1' a 'backup': File nuovo
DEBUG:root:File 'film.png' da copiare da 'folder1' a 'backup': File nuovo
DEBUG:root:File 'bunny.jpg' da copiare da 'folder1' a 'backup': File nuovo
DEBUG:root:Avvio processo 0: Process-1
DEBUG:root:Copia del file 'dog.jpg' da 'folder1' a 'backup'
DEBUG:root:Copia del file 'dog.jpg' da 'folder1' a 'backup' : Inizio
DEB